In [1]:
# Import PyAlex and other libraries
import pandas as pd
from embedding_atlas.projection import compute_text_projection
from embedding_atlas.widget import EmbeddingAtlasWidget

from pyalex import Works
from pyalex import invert_abstract

In [2]:
swin = (
    await Works()
    .filter_by_institution("https://openalex.org/I57093077")
    .get(limit=1000)
)

Output()

In [3]:
swin

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,...,referenced_works_count,referenced_works,related_works,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date,is_authors_truncated,fulltext_origin
0,https://openalex.org/W3160856016,https://doi.org/10.1016/j.jbusres.2021.04.070,How to conduct a bibliometric analysis: An ove...,How to conduct a bibliometric analysis: An ove...,2021,2021-05-14,{'openalex': 'https://openalex.org/W3160856016...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",article,...,72,"[https://openalex.org/W1021000864, https://ope...","[https://openalex.org/W4386031268, https://ope...",None,https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2025, 'cited_by_count': 2311}, {'yea...",2025-10-15T20:03:40.220052,2021-05-24,NaN,NaN
1,https://openalex.org/W2765081049,https://doi.org/10.1103/physrevlett.119.161101,GW170817: Observation of Gravitational Waves f...,GW170817: Observation of Gravitational Waves f...,2017,2017-10-16,{'openalex': 'https://openalex.org/W2765081049...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",article,...,190,"[https://openalex.org/W1134494461, https://ope...","[https://openalex.org/W4407587971, https://ope...","{'On': [0], 'August': [1], '17,': [2], '2017':...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2025, 'cited_by_count': 553}, {'year...",2025-10-18T13:47:04.470024,2017-11-10,True,pdf
2,https://openalex.org/W3135028703,https://doi.org/10.1007/s42979-021-00592-x,"Machine Learning: Algorithms, Real-World Appli...","Machine Learning: Algorithms, Real-World Appli...",2021,2021-03-22,{'openalex': 'https://openalex.org/W3135028703...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",review,...,148,"[https://openalex.org/W1128809682, https://ope...","[https://openalex.org/W4322629366, https://ope...",None,https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2025, 'cited_by_count': 983}, {'year...",2025-10-13T10:24:51.019780,2021-03-15,NaN,pdf
3,https://openalex.org/W2121951791,https://doi.org/10.1111/j.1365-2966.2011.19250.x,The 6dF Galaxy Survey: baryon acoustic oscilla...,The 6dF Galaxy Survey: baryon acoustic oscilla...,2011,2011-07-25,{'openalex': 'https://openalex.org/W2121951791...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",article,...,91,"[https://openalex.org/W1512335283, https://ope...","[https://openalex.org/W3100839555, https://ope...","{'We': [0, 36, 94, 169, 186, 227, 252], 'analy...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2025, 'cited_by_count': 136}, {'year...",2025-10-14T03:56:00.388177,2016-06-24,NaN,pdf
4,https://openalex.org/W2163266267,https://doi.org/10.1016/s1364-6613(00)01501-1,Social perception from visual cues: role of th...,Social perception from visual cues: role of th...,2000,2000-07-01,{'openalex': 'https://openalex.org/W2163266267...,en,"{'is_oa': False, 'landing_page_url': 'https://...",review,...,109,"[https://openalex.org/W1491642181, https://ope...","[https://openalex.org/W3014722448, https://ope...",None,https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2025, 'cited_by_count': 32}, {'year'...",2025-10-11T12:22:30.983026,2016-06-24,NaN,ngrams
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://openalex.org/W4289812837,https://doi.org/10.1016/j.conengprac.2022.105176,Electrochemical modeling and parameterization ...,Electrochemical modeling and parameterization ...,2022,2022-04-23,{'openalex': 'https://openalex.org/W4289812837...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,206,"[https://openalex.org/W150714319, https://open...","[https://openalex.org/W4388039502, https://ope...",None,https://api.openalex.org/works?filter=cites:W4...,"[{'year': 2025, 'cited_by_count': 27}, {'year'...",2025-10-16T07:16:01.360924,2022-08-05,NaN,pdf
996,https://openalex.org/W3202530004,https://doi.org/10.1021/jacs.1c06652,Plasmonic A

In [4]:
work_template = (
    lambda x: f"<work><title>{x['title']}</title><abstract>{invert_abstract(x['abstract_inverted_index'])}</abstract></work>"
)

In [5]:
swin["text"] = swin.apply(work_template, axis=1)

In [6]:
swin["subfield_name"] = pd.DataFrame(
    pd.DataFrame(swin.primary_topic.tolist()).subfield.tolist()
).display_name
swin["field_name"] = pd.DataFrame(
    pd.DataFrame(swin.primary_topic.tolist()).field.tolist()
).display_name

In [7]:
topics = pd.read_json("topics.json")
topic_template = (
    lambda x: f"<topic><name>{x['display_name']}</name><description>{x['description']}</description></topic>"
)
topics["subfield_name"] = topics.subfield.map(lambda x: x["display_name"])
topics["field_name"] = topics.field.map(lambda x: x["display_name"])
topics["text"] = topics.apply(topic_template, axis=1)

In [8]:
swin = swin[["id", "text", "subfield_name", "field_name"]]
topics = topics[["id", "text", "subfield_name", "field_name"]]

In [9]:
swin["type"] = "work"
topics["type"] = "topic"

/var/folders/k1/cn3zybm53kd95_crzqj9shg40000gn/T/ipykernel_92675/3862748297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  swin['type'] = "work"


In [10]:
df = pd.concat([topics, swin], axis=0)

In [11]:
compute_text_projection(df, text="text", x="projection_x", y="projection_y")

In [12]:
widget = EmbeddingAtlasWidget(
    df, x="projection_x", y="projection_y", labels="field_name"
)

In [13]:
widget

EmbeddingAtlasWidget()